In [1]:
import tkinter as tk
from tkinter import messagebox

class VendingMachine:
    def __init__(self):
        self.state = 'IDLE'
        self.stock = {'Soda': 2, 'Chips': 1, 'Candy': 0}
        self.selected_product = None

    def insert_coin(self):
        if self.state == 'IDLE':
            self.state = 'SELECTING_PRODUCT'
            return "Coin inserted. Please select a product."
        return "Coin already inserted or invalid state."

    def select_product(self, product):
        if self.state != 'SELECTING_PRODUCT':
            return "Insert a coin first."
        if product not in self.stock:
            return "Invalid product selected."
        if self.stock[product] == 0:
            self.state = 'OUT_OF_STOCK'
            return f"{product} is out of stock."
        self.selected_product = product
        self.state = 'DISPENSING'
        return f"{product} selected. Click 'Dispense' to get your product."

    def dispense(self):
        if self.state == 'DISPENSING':
            self.stock[self.selected_product] -= 1
            product_dispensed = self.selected_product
            self.selected_product = None
            self.state = 'IDLE'
            return f"{product_dispensed} dispensed. Thank you!"
        return "No product to dispense."

    def refill(self, product, amount):
        if product in self.stock:
            self.stock[product] += amount
            if self.state == 'OUT_OF_STOCK':
                self.state = 'IDLE'
            return f"{product} refilled by {amount}."
        return "Invalid product."

    def cancel(self):
        if self.state == 'SELECTING_PRODUCT':
            self.state = 'IDLE'
            return "Transaction cancelled."
        return "Nothing to cancel."

    def get_state(self):
        return self.state

    def get_stock(self):
        return self.stock.copy()

class VendingMachineGUI:
    def __init__(self, master):
        self.vm = VendingMachine()
        self.master = master
        master.title("Vending Machine")
        master.geometry("350x500")

        self.label = tk.Label(master, text="Welcome!", font=("Helvetica", 14))
        self.label.pack(pady=10)

        self.output = tk.Text(master, height=4, width=40, wrap=tk.WORD, bg="#f0f0f0")
        self.output.pack(pady=5)

        self.coin_button = tk.Button(master, text="Insert Coin", command=self.insert_coin, width=20, bg="lightblue")
        self.coin_button.pack(pady=5)

        tk.Label(master, text="Select Product:").pack()
        for product in ['Soda', 'Chips', 'Candy']:
            btn = tk.Button(master, text=product, width=20,
                            command=lambda p=product: self.select_product(p))
            btn.pack(pady=2)

        self.dispense_button = tk.Button(master, text="Dispense", command=self.dispense, width=20, bg="lightgreen")
        self.dispense_button.pack(pady=5)

        self.cancel_button = tk.Button(master, text="Cancel Transaction", command=self.cancel, width=20)
        self.cancel_button.pack(pady=5)

        self.stock_button = tk.Button(master, text="View Stock", command=self.show_stock, width=20)
        self.stock_button.pack(pady=5)

        self.refill_candy_button = tk.Button(master, text="Refill Candy", command=lambda: self.refill("Candy", 3), width=20)
        self.refill_candy_button.pack(pady=2)

        self.refill_soda_button = tk.Button(master, text="Refill Soda", command=lambda: self.refill("Soda", 3), width=20)
        self.refill_soda_button.pack(pady=2)

        self.refill_chips_button = tk.Button(master, text="Refill Chips", command=lambda: self.refill("Chips", 3), width=20)
        self.refill_chips_button.pack(pady=2)

        self.state_label = tk.Label(master, text="Current State: IDLE", font=("Helvetica", 12))
        self.state_label.pack(pady=10)

    def update_output(self, message):
        self.output.delete(1.0, tk.END)
        self.output.insert(tk.END, message)
        self.state_label.config(text=f"Current State: {self.vm.get_state()}")

    def insert_coin(self):
        result = self.vm.insert_coin()
        self.update_output(result)

    def select_product(self, product):
        result = self.vm.select_product(product)
        self.update_output(result)

    def dispense(self):
        result = self.vm.dispense()
        self.update_output(result)

    def cancel(self):
        result = self.vm.cancel()
        self.update_output(result)

    def show_stock(self):
        stock = self.vm.get_stock()
        message = "Current Stock:\n" + "\n".join(f"{k}: {v}" for k, v in stock.items())
        messagebox.showinfo("Stock Info", message)

    def refill(self, product, amount):
        result = self.vm.refill(product, amount)
        self.update_output(result)

if __name__ == '__main__':
    import sys
    if len(sys.argv) > 1 and sys.argv[1] == 'test':
        import unittest

        class TestVendingMachine(unittest.TestCase):
            def setUp(self):
                self.vm = VendingMachine()

            def test_initial_state(self):
                self.assertEqual(self.vm.get_state(), 'IDLE')

            def test_out_of_stock_product(self):
                self.vm.insert_coin()
                self.vm.select_product('Candy')  # Candy is out of stock
                self.assertEqual(self.vm.get_state(), 'OUT_OF_STOCK')

            def test_successful_dispense(self):
                self.vm.insert_coin()
                self.vm.select_product('Soda')
                self.vm.dispense()
                self.assertEqual(self.vm.get_state(), 'IDLE')
                self.assertEqual(self.vm.stock['Soda'], 1)

            def test_cancel_transaction(self):
                self.vm.insert_coin()
                self.vm.cancel()
                self.assertEqual(self.vm.get_state(), 'IDLE')

            def test_refill_product(self):
                self.vm.insert_coin()
                self.vm.select_product('Candy')  # Out of stock
                self.vm.refill('Candy', 2)
                self.assertEqual(self.vm.stock['Candy'], 2)
                self.assertEqual(self.vm.get_state(), 'IDLE')

        unittest.main(argv=['first-arg-is-ignored'], exit=False)
    else:
        root = tk.Tk()
        app = VendingMachineGUI(root)
        root.mainloop()